In [3]:
from PIL import Image
import torch
from torchvision.models import resnet18
from torchvision import transforms as T

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# input (single)
image = Image.open('cat.jpg')
transform = T.Compose([T.Resize((224, 224)), T.ToTensor()])
X = transform(image).unsqueeze(dim=0).to(device)

# original model
model = resnet18(weights="ResNet18_Weights.DEFAULT")
model = model.to(device)

# forward pass -- getting the outputs
out = model(X)

In [12]:
print(model)

tensor([463])
